# Connecting to a JBoss Virtual Database Using JDBC Driver

**NOTE:** In order for JayDeBeApi library to work with the Red Hat JBoss JDBC driver, have to install JPype1 version 0.6.3 instead of the latest version 0.7.  Otherwise, you'll get ```convertStrings``` error per this GitHub [issue](https://github.com/jpype-project/jpype/issues/498).

Therefore, you can install JayDeBeApi and jpype1 version 0.6.3 with conda command by doing:<br>
    ```conda install JayDeBeApi jpype1==0.6.3```

You also need to set the ```JAVA_HOME``` environment variable and know the path to the .jar file containing the teiid driver.

In [2]:
!echo %JAVA_HOME%

D:\jdk1_8


In [1]:
import jaydebeapi
import pandas as pd
from getpass import getpass

In [3]:
username = getpass("Enter your user name: ")
password = getpass("Enter your password: ")

Enter your user name: ········
Enter your password: ········


In [4]:
cnxn = jaydebeapi.connect("org.teiid.jdbc.TeiidDriver", 
                          "jdbc:teiid:cms_vdb_hna@mm://hamsxlpocdb01.some_company.com:31000",
                          [username, password],
                          # Alternative to setting the CLASSPATH environment variable, you can set the path with the jars option
                          jars='D:/LDE_Project/JDBC_Drivers/teiid-jdbc-8.12.14.6_4-redhat-64-3.jar')

In [5]:
sql = """
SELECT
    *
FROM
    cmsapp.SUPP_LOC_PLANT_TBL
"""

df = pd.read_sql(sql, cnxn, index_col=None)

cnxn.close()

In [6]:
df.head()

,ID,SUPP_NAME,SUPP_NO,SUPP_LOC_CODE,ALL_PLANTS,CREATED_DATE
0,7239928,METAL & WIRE PRODUCTS CO,012821,06,IPS,2020-01-09 01:00:04.380000
1,7239929,AISIN USA MFG INC,012829,Default,"ELP, HCM, HMA, HMI, IPS, MAP",2020-01-09 01:00:04.380000
2,7239930,None,012829,00,IPS,2020-01-09 01:00:04.380000
3,7239931,AISIN USA MFG INC,012829,01,"HCM, HMA",2020-01-09 01:00:04.380000
4,7239932,AISIN USA MFG INC,012829,02,"ELP, HMA, IPS, MAP",2020-01-09 01:00:04.380000


### Usually best practice to use a context manager when connecting to a database to ensure database connection will be closed

jaydebeapi library does not support context management, so we'll use contextmanager decorator from the contextlib module to enable it

In [7]:
from contextlib import contextmanager

In [10]:
@contextmanager
def db_connect(db, user, pwd):
    conn = jaydebeapi.connect('com.ibm.db2.jcc.DB2Driver', f'jdbc:teiid:{db}@mm://hamsxlpocdb01.some_company.com:31000',
                    [user, pwd], jars=['D:/JDBC_Drivers/mainframe_db2/db2jcc4.jar'])
    try:
        yield conn
    finally:
        conn.close()

In [11]:
with db_connect('cms_vdb_hna', username, password) as conn:
    df = pd.read_sql(sql, conn, index_col=None)

In [12]:
df.head()

,ID,SUPP_NAME,SUPP_NO,SUPP_LOC_CODE,ALL_PLANTS,CREATED_DATE
0,7239928,METAL & WIRE PRODUCTS CO,012821,06,IPS,2020-01-09 01:00:04.380000
1,7239929,AISIN USA MFG INC,012829,Default,"ELP, HCM, HMA, HMI, IPS, MAP",2020-01-09 01:00:04.380000
2,7239930,None,012829,00,IPS,2020-01-09 01:00:04.380000
3,7239931,AISIN USA MFG INC,012829,01,"HCM, HMA",2020-01-09 01:00:04.380000
4,7239932,AISIN USA MFG INC,012829,02,"ELP, HMA, IPS, MAP",2020-01-09 01:00:04.380000
